# Creating the Florida school number crosswalks

In [1]:
from os import path
import os

import numpy as np
import pandas as pd
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

/Users/maddy/Desktop/investigation-florida-online-attendance-master/venv/lib/python3.7/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


## Assert folders are in place

In [2]:
folders = [
    '../data/intermediary/keys/',
]

for folder in folders:
    if path.exists(folder):
        print("{folder} is already here!".format(folder=folder))
    else:
        try:
            os.makedirs(folder)
        except OSError:
            print("I couldn't make {folder}!".format(folder=folder))
        else:
            print("{folder} successfully made!".format(folder=folder))

../data/intermediary/keys/ successfully made!


## Broward

In [3]:
keys = pd.read_csv('../data/intermediary/scorecard/names/broward.csv')

In [4]:
raw_attendance = pd.read_csv('../data/input/foia/broward.csv')
attendance = pd.DataFrame()
attendance['school_name'] = raw_attendance['SchoolName'].unique()

In [5]:
choices = keys['school_name_l'].unique()

attendance['key_guess'] = attendance['school_name'].apply(lambda x: process.extract(x, choices, limit=1, scorer=fuzz.token_sort_ratio)[0])
attendance['guess_confidence'] = attendance['key_guess'].apply(lambda x: x[1])
attendance['key_guess'] = attendance['key_guess'].apply(lambda x: x[0])

Check rows that have a low guess confidence score:

In [6]:
attendance[attendance['guess_confidence'] < 75]

,school_name,key_guess,guess_confidence
128,NOVA HIGH,NOVA HIGH SCHOOL,72
153,PIPER HIGH,PIPER HIGH SCHOOL,74
216,WILLIAM T. MCFATTER TECHNICAL HIGH,MCFATTER TECHNICAL COLLEGE,68


Check and see if the amount unique guesses is equal to the number of schools:

In [7]:
print("key guesses:", attendance['key_guess'].nunique())
print("Actual district schools:", attendance['school_name'].nunique())
print("Off by:", attendance['school_name'].nunique() - attendance['key_guess'].nunique())

key guesses: 217
Actual district schools: 219
Off by: 2


List out which rows have duplicate guesses:

In [8]:
for school in attendance['key_guess'].unique():
    if (attendance[attendance['key_guess'] == school]['school_name'].nunique()) > 1:
        print("this guess used multiple times:",school)
        print('for these schools:')
        print(attendance[attendance['key_guess'].str.contains(school)]['school_name'].unique())
        print("\n")

this guess used multiple times: CYPRESS RUN ALTERNATIVE CENTER
for these schools:
['CYPRESS RUN ALTERNATIVE CENTER' 'PINE RIDGE ALTERNATIVE CENTER']


this guess used multiple times: PLANTATION PARK ELEMENTARY
for these schools:
['PLANTATION ELEMENTARY' 'PLANTATION PARK ELEMENTARY']




List out more potential guesses for those that need fixes:

In [9]:
need_fixes = ['PINE RIDGE ALTERNATIVE CENTER', 'PLANTATION ELEMENTARY']
for need in need_fixes:
    print(need)
    for guess in process.extract(need, choices, limit=10, scorer=fuzz.token_sort_ratio):
        print("\t", guess)

PINE RIDGE ALTERNATIVE CENTER
	 ('CYPRESS RUN ALTERNATIVE CENTER', 78)
	 ('PARK RIDGE ELEMENTARY SCHOOL', 49)
	 ('RENAISSANCE CHARTER SCHOOLS AT PINES', 49)
	 ('EAGLE RIDGE ELEMENTARY SCHOOL', 48)
	 ('CITY/PEMBROKE PINES CHARTER MIDDLE SCHOOL', 46)
	 ('PEMBROKE PINES CHARTER ELEMENTARY SCHOOL', 46)
	 ('WHIDDON RODGERS EDUCATION CENTER', 46)
	 ('EAGLE POINT ELEMENTARY SCHOOL', 45)
	 ('OAKRIDGE ELEMENTARY SCHOOL', 44)
	 ('PINEWOOD ELEMENTARY SCHOOL', 44)
PLANTATION ELEMENTARY
	 ('PLANTATION PARK ELEMENTARY', 89)
	 ('PLANTATION ELEMENTARY SCHOOL', 86)
	 ('WATKINS ELEMENTARY SCHOOL', 70)
	 ('FLORANADA ELEMENTARY SCHOOL', 67)
	 ('WINSTON PARK ELEMENTARY SCHOOL', 67)
	 ('MARGATE ELEMENTARY SCHOOL', 65)
	 ('FLAMINGO ELEMENTARY SCHOOL', 64)
	 ('LAKESIDE ELEMENTARY SCHOOL', 64)
	 ('LARKDALE ELEMENTARY SCHOOL', 64)
	 ('MANATEE BAY ELEMENTARY SCHOOL', 64)


Replace incorrect options:

In [10]:
school = 'PLANTATION ELEMENTARY'
fixed_guess = 'PLANTATION ELEMENTARY SCHOOL'

attendance['key_guess'] = np.where(attendance['school_name'] == school, fixed_guess, attendance['key_guess'])

Remove rows that don't seem to have matching data:

In [11]:
school = 'PINE RIDGE ALTERNATIVE CENTER'
attendance = attendance[attendance['school_name'] != school]

Save as a key file:

In [12]:
KEYS_FILENAME = '../data/intermediary/keys/broward.csv'
df = attendance.merge(keys, left_on='key_guess', right_on='school_name_l')
df[[
    'school_name', 
    'key_guess', 
    'guess_confidence', 
    'district_number', 
    'district_name', 
    'school_number',
    'school_name_l'
]].to_csv(KEYS_FILENAME, index=False)